In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

In [2]:
from fastai2.torch_core import *
from fastai2.data.all import *
from fastai2.learner import *
from fastai2.metrics import *
from fastai2.callback.all import *

In [3]:
from timeseries.imports import *
from timeseries.utils import *
from timeseries.data import *
from timeseries.core import *
from timeseries.models import *

In [4]:
def cycle_dl(dl):
    for x,y in iter(dl): pass

In [34]:
X_1000_2_10_in_memory = np.random.rand(1000, 2, 10).astype('float32')
np.save('../data/X_1000_2_10.npy', X_1000_2_10_in_memory)
X_1000_2_10_on_disk = np.load('../data/X_1000_2_10.npy', mmap_mode='r')

X_1000_20_100_in_memory = np.random.rand(1000, 20, 100).astype('float32')
np.save('../data/X_1000_20_100.npy', X_1000_20_100_in_memory)
X_1000_20_100_on_disk = np.load('../data/X_1000_20_100.npy', mmap_mode='r')

X_1000_2_1000_in_memory = np.random.rand(1000, 2, 1000).astype('float32')
np.save('../data/X_1000_2_1000.npy', X_1000_2_1000_in_memory)
X_1000_2_1000_on_disk = np.load('../data/X_1000_2_1000.npy', mmap_mode='r')

X_10000_2_10_in_memory = np.random.rand(10000, 2, 10).astype('float32')
np.save('../data/X_10000_2_10.npy', X_10000_2_10_in_memory)
X_10000_2_10_on_disk = np.load('../data/X_10000_2_10.npy', mmap_mode='r')

# StarLight valid like
X_1000_1_1024_in_memory = np.random.rand(1000, 1, 1024).astype('float32')
np.save('../data/X_1000_1_1024.npy', X_1000_1_1024_in_memory)
X_8236_1_1024_on_disk = np.load('../data/X_8236_1_1024.npy', mmap_mode='r')

X_8236_1_1024_in_memory = np.random.rand(8236, 1, 1024).astype('float32')
np.save('../data/X_8236_1_1024.npy', X_8236_1_1024_in_memory)
X_8236_1_1024_on_disk = np.load('../data/X_8236_1_1024.npy', mmap_mode='r')

X_starlight_in_memory = concat(X_1000_1_1024_in_memory, X_8236_1_1024_in_memory)
np.save('../data/X_starlight.npy', X_starlight_in_memory)
X_starlight_on_disk = np.load('../data/X_starlight.npy', mmap_mode='r')
splits = (L(list(np.arange(len(X_1000_1_1024_in_memory)))), L(list(np.arange(len(X_1000_1_1024_in_memory), len(X_starlight_in_memory)))))


y_1000 = np.random.randint(0, 10, 1000)
y_10000 = np.random.randint(0, 10, 10000)
y_8236 = np.random.randint(0, 10, 8236)
y_starlight = concat(y_1000, y_8236)

In [8]:
# Native Pytorch
class NumpyDataset(torch.utils.data.Dataset):
    def __init__(self, X, y=None, sel_vars=None, sel_steps=None):
        self.X, self.y = X, y
        self.sel_vars =  slice(None) if sel_vars is None else sel_vars
        self.sel_steps =  slice(None) if sel_steps is None else sel_steps

    def __getitem__(self, idx):
        if self.y is None: return (self.X[idx, self.sel_vars, self.sel_steps], )
        else: return (self.X[idx, self.sel_vars, self.sel_steps], self.y[idx])

    def __len__(self):
        return len(self.X)

### Baseline: Native Pytorch

In [9]:
dset    = NumpyDataset(X_1000_2_10_in_memory, y_1000)
dloader = torch.utils.data.DataLoader(dataset=dset, batch_size=128, shuffle=True)
xb,yb   = next(iter(dloader))
print(xb.shape, yb.shape)
%timeit cycle_dl(dloader)

torch.Size([128, 2, 10]) torch.Size([128])
28.4 ms ± 211 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
dset    = NumpyDataset(X_1000_20_100_in_memory, y_1000)
dloader = torch.utils.data.DataLoader(dataset=dset, batch_size=128, shuffle=True)
xb,yb   = next(iter(dloader))
print(xb.shape, yb.shape)
%timeit cycle_dl(dloader)

torch.Size([128, 20, 100]) torch.Size([128])
35.7 ms ± 4.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
dset    = NumpyDataset(X_1000_2_1000_in_memory, y_1000)
dloader = torch.utils.data.DataLoader(dataset=dset, batch_size=128, shuffle=True)
xb,yb   = next(iter(dloader))
print(xb.shape, yb.shape)
%timeit cycle_dl(dloader)

torch.Size([128, 2, 1000]) torch.Size([128])
43.7 ms ± 5.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
dset    = NumpyDataset(X_10000_2_10_in_memory, y_10000)
dloader = torch.utils.data.DataLoader(dataset=dset, batch_size=128, shuffle=True)
xb,yb   = next(iter(dloader))
print(xb.shape, yb.shape)
%timeit cycle_dl(dloader)

torch.Size([128, 2, 10]) torch.Size([128])
434 ms ± 66.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
# test larger batch size
dset    = NumpyDataset(X_10000_2_10_in_memory, y_10000)
dloader = torch.utils.data.DataLoader(dataset=dset, batch_size=1280, shuffle=True)
xb,yb   = next(iter(dloader))
print(xb.shape, yb.shape)
%timeit cycle_dl(dloader)

torch.Size([1280, 2, 10]) torch.Size([1280])
363 ms ± 36.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Learning:** The number of dimensions or length of the time series doesn't have a significant impact on dataloader's performance. Performance is mostly determined by the number of samples, not the number of batches.

In [14]:
# StarLight valid like
dset    = NumpyDataset(X_8236_1_1024_in_memory, y_8236)
dloader = torch.utils.data.DataLoader(dataset=dset, batch_size=128, shuffle=True)
xb,yb   = next(iter(dloader))
print(xb.shape, yb.shape)
%timeit cycle_dl(dloader)

torch.Size([128, 1, 1024]) torch.Size([128])
265 ms ± 20.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
def dataloader(dataset,
               batch_size=1,
               shuffle=False,
               sampler=None,
               batch_sampler=None,
               num_workers=0,
               collate_fn=np_collate,
               pin_memory=False,
               drop_last=False,
               timeout=0,
               worker_init_fn=None,
               multiprocessing_context=None):

    return torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        sampler=sampler,
        batch_sampler=batch_sampler,
        num_workers=num_workers,
        collate_fn=collate_fn,
        pin_memory=pin_memory,
        drop_last=drop_last,
        timeout=timeout,
        worker_init_fn=worker_init_fn,
        multiprocessing_context=multiprocessing_context)


NumpyDataset.dataloader = dataloader

In [16]:
dset    = NumpyDataset(X_1000_20_100_in_memory, y_1000)
dloader = dset.dataloader(batch_size=128)
xb,yb   = next(iter(dloader))
xb.shape

torch.Size([128, 20, 100])

The updated collate function that allows memmap conversion into a tensor doesn't negetively impact performance, while it allows you to pass np.memmap to the dataloader without previous conversion to tensor or np.array. There is a slight delay however if data is on disk vs in memory, so it's better to pass data in memory if possible.

In [17]:
dset    = NumpyDataset(X_1000_20_100_in_memory, y_1000)
dloader = dset.dataloader(batch_size=128, shuffle=True)
xb,yb   = next(iter(dloader))
print(xb.shape, yb.shape)
%timeit cycle_dl(dloader)

torch.Size([128, 20, 100]) torch.Size([128])
36.8 ms ± 6.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
dset    = NumpyDataset(X_1000_20_100_on_disk, y_1000)
dloader = dset.dataloader(batch_size=128, shuffle=True)
xb,yb   = next(iter(dloader))
print(xb.shape, yb.shape)
%timeit cycle_dl(dloader)

torch.Size([128, 20, 100]) torch.Size([128])
41.7 ms ± 416 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
dset    = NumpyDataset(X_1000_2_1000_in_memory, y_1000)
dloader = dset.dataloader(batch_size=128, shuffle=True)
xb,yb   = next(iter(dloader))
print(xb.shape, yb.shape)
%timeit cycle_dl(dloader)

torch.Size([128, 2, 1000]) torch.Size([128])
29.9 ms ± 185 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
dset    = NumpyDataset(X_1000_2_1000_on_disk, y_1000)
dloader = dset.dataloader(batch_size=128, shuffle=True)
xb,yb   = next(iter(dloader))
print(xb.shape, yb.shape)
%timeit cycle_dl(dloader)

torch.Size([128, 2, 1000]) torch.Size([128])
51.5 ms ± 4.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
dset    = NumpyDataset(X_10000_2_10_in_memory, y_10000)
dloader = dset.dataloader(batch_size=128, shuffle=True)
xb,yb   = next(iter(dloader))
print(xb.shape, yb.shape)
%timeit cycle_dl(dloader)

torch.Size([128, 2, 10]) torch.Size([128])
315 ms ± 27.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
dset    = NumpyDataset(X_10000_2_10_on_disk, y_10000)
dloader = dset.dataloader(batch_size=128, shuffle=True)
xb,yb   = next(iter(dloader))
print(xb.shape, yb.shape)
%timeit cycle_dl(dloader)

torch.Size([128, 2, 10]) torch.Size([128])
455 ms ± 35.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
# StarLight valid like
dset    = NumpyDataset(X_8236_1_1024_in_memory, y_8236)
dloader = dset.dataloader(batch_size=128, shuffle=True)
xb,yb   = next(iter(dloader))
print(xb.shape, yb.shape)
%timeit cycle_dl(dloader)

torch.Size([128, 1, 1024]) torch.Size([128])
286 ms ± 26.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
# StarLight valid like
dset    = NumpyDataset(X_8236_1_1024_on_disk, y_8236)
dloader = dset.dataloader(batch_size=128, shuffle=True)
xb,yb   = next(iter(dloader))
print(xb.shape, yb.shape)
%timeit cycle_dl(dloader)

torch.Size([128, 1, 1024]) torch.Size([128])
351 ms ± 18.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
# conversion to items
class NumpyDataset2(torch.utils.data.Dataset):
    def __init__(self, X, y=None):
        self.items = itemify(X,) if y is None else itemify(X,y)

    def __getitem__(self, idx):
        return self.items[idx]

    def __len__(self):
        return len(self.items)
    
NumpyDataset2.dataloader = dataloader

Conversion to items doesn't improve performance. Performance is basically the same, so it's not worth it.

In [26]:
dset    = NumpyDataset2(X_1000_20_100_in_memory, y_1000)
dloader = dset.dataloader(batch_size=128, shuffle=True)
xb,yb = next(iter(dloader))
print(xb.shape, yb.shape)
%timeit cycle_dl(dloader)

torch.Size([128, 20, 100]) torch.Size([128])
31.4 ms ± 1.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
#conversion to TS tensor
class TSDataset(torch.utils.data.Dataset):
    def __init__(self, X, y=None):
        self.X, self.y = X, y
        
    def __getitem__(self, idx):
        if self.y is None: return (self.X[idx], )
        else: return (TSTensor(self.X[idx]), self.y[idx])

    def __len__(self):
        return len(self.X)
    
TSDataset.dataloader = dataloader

Conversion to TSTensor decreases performance, while it doesn't bring any significant benefit

In [28]:
dset    = TSDataset(X_1000_20_100_in_memory, y_1000)
dloader = dset.dataloader(batch_size=128, shuffle=True)
xb,yb = next(iter(dloader))
print(xb.shape, yb.shape)
%timeit cycle_dl(dloader)

57.7 ms ± 15.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Factory method

In [36]:
dls = TSDataLoaders.from_numpy(X_starlight_in_memory, y_starlight, splitter=IndexSplitter(splits[1]), bs=64, val_bs=128)
xb,yb = next(iter(dls.valid))
print(xb.shape, yb.shape)
%time cycle_dl(dls.valid)

torch.Size([128, 1, 1024]) torch.Size([128])
CPU times: user 159 ms, sys: 156 ms, total: 315 ms
Wall time: 2.82 s


### Datablock

In [39]:
getters = [ItemGetter(0), ItemGetter(1)]
dblock = DataBlock(blocks=(TSTensorBlock, CategoryBlock),
                   getters=getters,
                   splitter=IndexSplitter(splits[1]),
                   item_tfms=None,
                   batch_tfms=None)
source = itemify(X_starlight_in_memory, y_starlight)
dls = dblock.dataloaders(source, bs=64, val_bs=128)
xb,yb = next(iter(dls.valid))
print(xb.shape, yb.shape)
%time cycle_dl(dls.valid)

torch.Size([128, 1, 1024]) torch.Size([128])
CPU times: user 166 ms, sys: 153 ms, total: 319 ms
Wall time: 3.5 s


### TSDatasets + TSDataLoaders

In [46]:
class NumpyDatasets(FilteredBase):
    "A dataset that creates a tuple from each `tfms`, passed thru `item_tfms`"
    def __init__(self, X=None, y=None, items=None, tfms=None, tls=None, n_inp=None, dl_type=None, **kwargs):
        super().__init__(dl_type=dl_type)

        #New 4 lines of code
        if tls is None:
            if items is None: items = itemify(X,) if y is None else itemify(X,y)
            assert (tfms is None or len(items[0]) == len(tfms)), f"n_tfms ({len(tfms)}) doesn't match n_items ({len(items[0])})"
            tfms = None if tfms is None else [[ItemGetter(i)] + L(tfms[i]) for i in range(len(items[0]))]
        
        self.tls = L(tls if tls else [TfmdLists(items, t, **kwargs) for t in L(ifnone(tfms,[None]))])
        self.n_inp = (1 if len(self.tls)==1 else len(self.tls)-1) if n_inp is None else n_inp

    def __getitem__(self, it):
        res = tuple([tl[it] for tl in self.tls])
        return res if is_indexer(it) else list(zip(*res))

    def __getattr__(self,k): return gather_attrs(self, k, 'tls')
    def __dir__(self): return super().__dir__() + gather_attr_names(self, 'tls')
    def __len__(self): return len(self.tls[0])
    def __iter__(self): return (self[i] for i in range(len(self)))
    def __repr__(self): return coll_repr(self)
    def decode(self, o, full=True): return tuple(tl.decode(o_, full=full) for o_,tl in zip(o,tuplify(self.tls, match=o)))
    def subset(self, i): return type(self)(tls=L(tl.subset(i) for tl in self.tls), n_inp=self.n_inp)
    def _new(self, items, *args, **kwargs): return super()._new(items, tfms=self.tfms, do_setup=False, **kwargs)
    def overlapping_splits(self): return self.tls[0].overlapping_splits()
    @property
    def splits(self): return self.tls[0].splits
    @property
    def split_idx(self): return self.tls[0].tfms.split_idx
    @property
    def items(self): return self.tls[0].items
    @items.setter
    def items(self, v):
        for tl in self.tls: tl.items = v

    def show(self, o, ctx=None, **kwargs):
        for o_,tl in zip(o,self.tls): ctx = tl.show(o_, ctx=ctx, **kwargs)
        return ctx

    def new_empty(self):
        tls = [tl._new([], split_idx=tl.split_idx) for tl in self.tls]
        return type(self)(tls=tls, n_inp=self.n_inp)

    @contextmanager
    def set_split_idx(self, i):
        old_split_idx = self.split_idx
        for tl in self.tls: tl.tfms.split_idx = i
        yield self
        for tl in self.tls: tl.tfms.split_idx = old_split_idx

In [52]:
tfms = [[], [Categorize()]]
dsets = NumpyDatasets(X_starlight_in_memory, y_starlight, tfms=tfms, splits=splits)
dls = TSTfmdDL(dsets, bs=64, val_bs=128)
xb,yb = next(iter(dls.valid))
print(xb.shape, yb.shape)
%time cycle_dl(dls.valid)

(1, 1024) torch.Size([])
CPU times: user 5.72 s, sys: 21.5 ms, total: 5.74 s
Wall time: 5.74 s


In [50]:
dls = TSTfmdDL(dsets, bs=64, val_bs=128)
xb,yb = next(iter(dls.valid))
xb, yb

(TSTensor(vars:1, len:1024), TensorCategory(0))

In [51]:
xb, yb = dls.one_batch()
xb, yb

(TSTensor(samples:64, vars:1, len:1024),
 TensorCategory([6, 7, 1, 4, 0, 7, 7, 4, 6, 0, 0, 8, 9, 2, 6, 4, 1, 3, 9, 1, 9, 5, 2, 7,
         7, 9, 4, 6, 0, 6, 4, 6, 6, 2, 2, 8, 4, 1, 8, 7, 2, 0, 9, 9, 6, 5, 7, 5,
         8, 0, 8, 2, 6, 1, 6, 2, 8, 8, 4, 0, 4, 1, 7, 2]))

In [56]:
np_collate

<function timeseries.data.np_collate(batch)>